In [1]:
import pandas as pd
import numpy as np
import os
import sys
import gzip
import shutil
import json
import re
import argparse
import requests
import Bio.PDB as bpdb
from Bio import Align
import tmalign_mapping

/home/srayan/miniconda3/envs/idp/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/tmp/ipykernel_237732/1172264817.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
uniprot_id = "PLBL2_MOUSE"

In [7]:
uniprot_pdb_path = "/nfs/turbo/lsa-tewaria/uniprot_df_small.csv"
uniprot_df = pd.read_csv(uniprot_pdb_path)

In [8]:
uniprot_json = tmalign_mapping.find_uniprot(uniprot_df, uniprot_id)

In [9]:
main_sequence = tmalign_mapping.extract_sequence(uniprot_json)

In [17]:
domain_info = pd.read_csv("/nfs/turbo/lsa-tewaria/uniprot/PLBL2_MOUSE/domain_info.csv", index_col=0)
print(domain_info)

          name  start  end  d_8046668
0  3fgr_0A.pdb     62  238      False
1  3fgr_0B.pdb    244  248      False
2  3fgr_0C.pdb    249  592      False
3  3fgt_0A.pdb     59  239      False
4  3fgt_0B.pdb    249  592      False
5  3fgw_0A.pdb     60  239      False
6  3fgw_0B.pdb    244  594      False


In [13]:
with open("/nfs/turbo/lsa-tewaria/uniprot/PLBL2_MOUSE/PLBL2_MOUSE.fasta", "w") as f:
    for x in range(len(domain_info["name"])):
        start = domain_info.iloc[x]["start"]
        end = domain_info.iloc[x]["end"]
        f.write(f">{domain_info.iloc[x]['name']}\n")
        f.write(main_sequence[start:end] + "\n")

In [22]:
uniprot_path = "/nfs/turbo/lsa-tewaria/uniprot/PLBL2_MOUSE"
fasta_path = "/nfs/turbo/lsa-tewaria/uniprot/PLBL2_MOUSE/PLBL2_MOUSE.fasta"
terminal_command = "mmseqs easy-cluster " + fasta_path + " " + os.path.join(uniprot_path, "clusterRes") + " tmp --min-seq-id 0.8 -c 0.8 --cov-mode 1"

In [23]:
os.system(terminal_command)

easy-cluster /nfs/turbo/lsa-tewaria/uniprot/PLBL2_MOUSE/PLBL2_MOUSE.fasta /nfs/turbo/lsa-tewaria/uniprot/PLBL2_MOUSE/clusterRes tmp --min-seq-id 0.8 -c 0.8 --cov-mode 1 

MMseqs Version:                     	13.45111
Substitution matrix                 	nucl:nucleotide.out,aa:blosum62.out
Seed substitution matrix            	nucl:nucleotide.out,aa:VTML80.out
Sensitivity                         	4
k-mer length                        	0
k-score                             	2147483647
Alphabet size                       	nucl:5,aa:21
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.8
Coverage mode                       	1
Compositional bias                  	1
Diagonal scoring                    	true
Exact k-mer matching                	0
Mask residues                       	1
Mask lower case residues

0

In [28]:
cluster_put = pd.read_csv(os.path.join(uniprot_path, "clusterRes_cluster.tsv"), sep="\t", header=None, names=["cluster", "pdb"])

In [29]:
print(cluster_put)

       cluster          pdb
0  3fgr_0B.pdb  3fgr_0B.pdb
1  3fgt_0A.pdb  3fgt_0A.pdb
2  3fgt_0A.pdb  3fgr_0A.pdb
3  3fgt_0A.pdb  3fgw_0A.pdb
4  3fgw_0B.pdb  3fgw_0B.pdb
5  3fgw_0B.pdb  3fgr_0C.pdb
6  3fgw_0B.pdb  3fgt_0B.pdb
